In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
import string

## Pre-process some tokens

In [2]:
words = pd.read_csv('words.csv')

IOError: File words.csv does not exist

In [14]:
words.ix[:,0]

0          adult 
1          sexual
2             sex
3       sexuality
4        romantic
5      lovemaking
6            kids
7           crush
8          orgasm
9          seduce
10          sperm
11        bladder
12         female
13        fashion
14         couple
15     girlfriend
16      cosmetics
17         makeup
18       makeover
19      boyfriend
20    accessories
21        jewelry
22            bff
23        cleanse
24      exfoliate
25     moisturize
26       warning 
27      addicting
28      princess 
29          dress
30          party
31         dating
32          strip
33           lust
34         lovers
35         erotic
36          boobs
37            gay
38           porn
39          penis
40            hot
41           nude
42          naked
43           kiss
44           lady
45          salon
46       painting
47           skin
48        mermaid
49        trained
50         sniper
51            gun
52         attact
Name: ﻿word, dtype: object

In [18]:
l = words.ix[:,0].tolist()
s = ' '.join(l)

In [19]:
token1=nltk.word_tokenize(s)

In [22]:
token1

[u'adult',
 u'sexual',
 u'sex',
 u'sexual',
 u'romant',
 u'lovemak',
 u'kid',
 u'crush',
 u'orgasm',
 u'seduc',
 u'sperm',
 u'bladder',
 u'femal',
 u'fashion',
 u'coupl',
 u'girlfriend',
 u'cosmet',
 u'makeup',
 u'makeov',
 u'boyfriend',
 u'accessori',
 u'jewelri',
 u'bff',
 u'cleans',
 u'exfoli',
 u'moistur',
 u'warn',
 u'addict',
 u'princess',
 u'dress',
 u'parti',
 u'date',
 u'strip',
 u'lust',
 u'lover',
 u'erot',
 u'boob',
 u'gay',
 u'porn',
 u'peni',
 u'hot',
 u'nude',
 u'nake',
 u'kiss',
 u'ladi',
 u'salon',
 u'paint',
 u'skin',
 u'mermaid',
 u'train',
 u'sniper',
 u'gun',
 u'attact']

In [21]:
porter = nltk.PorterStemmer()
token1 = [porter.stem(w) for w in token1]

In [23]:
words['token'] = token1

In [24]:
words.to_csv('words+tokens.csv')

## Find the distribution of Strong tokens in each level

### Read "Strong" tokens

In [84]:
strong = pd.read_excel('strong_words.xlsx')

In [85]:
strong.head()

,token
0,adult
1,sex
2,sexual
3,romant
4,lovemak


In [86]:
lst = strong['token'].tolist()

In [87]:
result = pd.DataFrame(data = 0,index = range(len(lst)),columns=['token','1','2','3','4'])

In [88]:
result['token'] = lst

In [89]:
result

,token,1,2,3,4
0,adult,0,0,0,0
1,sex,0,0,0,0
2,sexual,0,0,0,0
3,romant,0,0,0,0
4,lovemak,0,0,0,0
5,spa,0,0,0,0
6,bridal,0,0,0,0
7,flirti,0,0,0,0
8,foreplay,0,0,0,0
9,orgasm,0,0,0,0


### Preprocess whole app dataset

In [46]:
df = pd.read_csv('Whole_Androiddata.csv')

In [51]:
mystopwords = stopwords.words('english')
def cleandata(row):
    des_list = row['description'].split(' ')
    #delete web links 
    new_list = [word.lower() for word in des_list if not word.startswith('www') and not word.startswith('http')]
    new_des = ' '.join(new_list)
    
    #remove useless punctuation 
    new_des = new_des.replace('\'','').replace('\"','').replace('~','')
    
    #mark other language descriptions
    mystopwords = stopwords.words('english')
    flag = 0
    for w in mystopwords:
        if w in new_list:
            flag = 1
    
    row['isEnglish'] = flag
    row['new description'] = new_des
    
    return row

In [52]:
df = df.apply(cleandata,axis=1)
df = df[df['isEnglish']==1]

In [53]:
def process(row):
    
    ## Tokenize
    tokens = nltk.word_tokenize(row['new description'])
    
    ## stemming, remove stopwords
    porter = nltk.PorterStemmer()
    tokens2 = [porter.stem(w) for w in tokens]
    tokens3 = [w for w in tokens2 if w not in string.punctuation and w not in mystopwords]

    row['tokens']=tokens3
    
    ## word count
    row['word_count'] = len(tokens3)
    
    return row

In [54]:
df = df.apply(process,axis=1)
df

,name,sub_category,maturity_rating,maturity_rating_reason,download_num,developer,developer_address,price,datePublished,version,review_score,review_num,app_url,icon_url,description,isEnglish,new description,tokens,word_count
0,10.5,Card,Teen,Simulated Gambling,"100,000 - 500,000",xidea creator,"8F., No.174, Sec. 2, Nanjing E. Rd., Zhongshan...",NaN,5-Sep-16,1.0.7,3.403496,1373.0,https://play.google.com/store/apps/details?id=...,http:https://lh5.ggpht.com/VyiwSUaNyE19EpmDOsP...,(Introduction) Ten Point Half(10.5) is a very ...,1,(introduction) ten point half(10.5) is a very ...,"[introduct, ten, point, half, 10.5, veri, simp...",118
1,101,Card,Everyone,NaN,"50,000 - 100,000",Frigate Studios,NaN,NaN,1-Dec-16,1.23,4.312939,997.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/mvneCoXs-pg2n...,"There are many variations of the basic game, a...",1,"there are many variations of the basic game, a...","[mani, variat, basic, game, number, differ, na...",42
2,2017,Adventure,Everyone,NaN,10 - 50,med dev,NaN,NaN,29-Dec-16,1,5.000000,3.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/-r9zhCalegAqw...,Hello you in your game Bunny helped Collect co...,1,hello you in your game bunny helped collect co...,"[hello, game, bunni, help, collect, coin, coll...",10
3,2017,Sports,Everyone,NaN,100 - 500,MYLAPS Experience Lab,Overschiestraat 65 1062 XD Amsterdam The Nethe...,NaN,10-Feb-17,1,4.230769,13.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/F_TGml00P7NBn...,The 2017 mobile app is the most complete app f...,1,the 2017 mobile app is the most complete app f...,"[2017, mobil, app, complet, app, ultim, event,...",39
4,2048,Puzzle,Everyone,NaN,50 - 100,prank call puzzle,NaN,NaN,12-Aug-16,1,NaN,NaN,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/hVrtRVeoy_-yk...,2048 game is a funnysimple and addictive numbe...,1,2048 game is a funnysimple and addictive numbe...,"[2048, game, funnysimpl, addict, number, puzzl...",101
5,2048,Puzzle,Everyone,NaN,10 - 50,iCare Edutainment,"D05, 28/F, TML Tower, No. 3 Hoi Shing Road, Ts...",NaN,5-Feb-17,1.3,NaN,NaN,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/PqkgGplcxr3qF...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,1,2048 game tips & tricks 1. take it slow 2048 i...,"[2048, game, tip, trick, 1., take, slow, 2048,...",145
6,2048,Puzzle,Unrated,Warning ��� content has not yet been rated. Un...,"10,000 - 50,000",NG-TEAM,NaN,NaN,8-Jun-14,2.2,4.135831,427.0,https://play.google.com/store/apps/details?id=...,http://lh4.ggpht.com/IHXcAk-DG1WmAJhVNcUT_9O2E...,2048 is a simple but very attractive math puzz...,1,2048 is a simple but very attractive math puzz...,"[2048, simpl, veri, attract, math, puzzl, game...",60
7,2048,Puzzle,Everyone,NaN,100 - 500,hciLab,University of Stuttgart Human-Computer Interac...,NaN,27-Feb-17,0.4,4.384615,13.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/R-ildyLUXKs6I...,2048 is a singleplayer sliding block puzzle game.,1,2048 is a singleplayer sliding block puzzle game.,"[2048, singleplay, slide, block, puzzl, game]",6
8,2048,Puzzle,Everyone,NaN,"10,000 - 50,000",Bazinga!,"No. 477 , Aomen Road Putuo Area,Shanghai,China",NaN,21-Jun-14,1.02,4.351942,824.0,https://play.google.com/store/apps/details?id=...,http://lh4.ggpht.com/mF_LE80-JtYPh4QhdlqTQ7xxe...,"2048 is a super addictive brain game , try to ...",1,"2048 is a super addictive brain game , try to ...","[2048, super, addict, brain, game, tri, merg, ...",145
9,2048,Strategy,Everyone,NaN,"50,000 - 100,000",WANGJIE,10 Rue Jean Baptiste Charcot,NaN,29-Apr-16,1.3,4.540123,324.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/V6BDyrFCGp_tj...,"2048 is played on a gray 44 grid, with numbere...",1,"2048 is played on a gray 44 grid, with numbere...","[2048, play, gray, 44, grid, number, tile, sli...",

In [55]:
# convert label to a consecutive numerical variabel
df['level'] = df.maturity_rating.map({'Everyone':1,'Everyone 10+':2,'Teen':3,'Mature 17+':4})
df

,name,sub_category,maturity_rating,maturity_rating_reason,download_num,developer,developer_address,price,datePublished,version,review_score,review_num,app_url,icon_url,description,isEnglish,new description,tokens,word_count,level
0,10.5,Card,Teen,Simulated Gambling,"100,000 - 500,000",xidea creator,"8F., No.174, Sec. 2, Nanjing E. Rd., Zhongshan...",NaN,5-Sep-16,1.0.7,3.403496,1373.0,https://play.google.com/store/apps/details?id=...,http:https://lh5.ggpht.com/VyiwSUaNyE19EpmDOsP...,(Introduction) Ten Point Half(10.5) is a very ...,1,(introduction) ten point half(10.5) is a very ...,"[introduct, ten, point, half, 10.5, veri, simp...",118,3.0
1,101,Card,Everyone,NaN,"50,000 - 100,000",Frigate Studios,NaN,NaN,1-Dec-16,1.23,4.312939,997.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/mvneCoXs-pg2n...,"There are many variations of the basic game, a...",1,"there are many variations of the basic game, a...","[mani, variat, basic, game, number, differ, na...",42,1.0
2,2017,Adventure,Everyone,NaN,10 - 50,med dev,NaN,NaN,29-Dec-16,1,5.000000,3.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/-r9zhCalegAqw...,Hello you in your game Bunny helped Collect co...,1,hello you in your game bunny helped collect co...,"[hello, game, bunni, help, collect, coin, coll...",10,1.0
3,2017,Sports,Everyone,NaN,100 - 500,MYLAPS Experience Lab,Overschiestraat 65 1062 XD Amsterdam The Nethe...,NaN,10-Feb-17,1,4.230769,13.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/F_TGml00P7NBn...,The 2017 mobile app is the most complete app f...,1,the 2017 mobile app is the most complete app f...,"[2017, mobil, app, complet, app, ultim, event,...",39,1.0
4,2048,Puzzle,Everyone,NaN,50 - 100,prank call puzzle,NaN,NaN,12-Aug-16,1,NaN,NaN,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/hVrtRVeoy_-yk...,2048 game is a funnysimple and addictive numbe...,1,2048 game is a funnysimple and addictive numbe...,"[2048, game, funnysimpl, addict, number, puzzl...",101,1.0
5,2048,Puzzle,Everyone,NaN,10 - 50,iCare Edutainment,"D05, 28/F, TML Tower, No. 3 Hoi Shing Road, Ts...",NaN,5-Feb-17,1.3,NaN,NaN,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/PqkgGplcxr3qF...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,1,2048 game tips & tricks 1. take it slow 2048 i...,"[2048, game, tip, trick, 1., take, slow, 2048,...",145,1.0
6,2048,Puzzle,Unrated,Warning ��� content has not yet been rated. Un...,"10,000 - 50,000",NG-TEAM,NaN,NaN,8-Jun-14,2.2,4.135831,427.0,https://play.google.com/store/apps/details?id=...,http://lh4.ggpht.com/IHXcAk-DG1WmAJhVNcUT_9O2E...,2048 is a simple but very attractive math puzz...,1,2048 is a simple but very attractive math puzz...,"[2048, simpl, veri, attract, math, puzzl, game...",60,NaN
7,2048,Puzzle,Everyone,NaN,100 - 500,hciLab,University of Stuttgart Human-Computer Interac...,NaN,27-Feb-17,0.4,4.384615,13.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/R-ildyLUXKs6I...,2048 is a singleplayer sliding block puzzle game.,1,2048 is a singleplayer sliding block puzzle game.,"[2048, singleplay, slide, block, puzzl, game]",6,1.0
8,2048,Puzzle,Everyone,NaN,"10,000 - 50,000",Bazinga!,"No. 477 , Aomen Road Putuo Area,Shanghai,China",NaN,21-Jun-14,1.02,4.351942,824.0,https://play.google.com/store/apps/details?id=...,http://lh4.ggpht.com/mF_LE80-JtYPh4QhdlqTQ7xxe...,"2048 is a super addictive brain game , try to ...",1,"2048 is a super addictive brain game , try to ...","[2048, super, addict, brain, game, tri, merg, ...",145,1.0
9,2048,Strategy,Everyone,NaN,"50,000 - 100,000",WANGJIE,10 Rue Jean Baptiste Charcot,NaN,29-Apr-16,1.3,4.540123,324.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/V6BDyrFCGp_tj...,"2048 is played on a gray 44 grid, with numbere...",1,"2048 is played on a gray 44 grid, with numbere...","[2048, pl

In [56]:
df.to_csv('Android_processed_0314.csv')

In [64]:
len(df)

36927

In [62]:
b_list = ['sex' in token_list for token_list in df['tokens']]

In [65]:
df[df['level']==1]['tokens']

1        [mani, variat, basic, game, number, differ, na...
2        [hello, game, bunni, help, collect, coin, coll...
3        [2017, mobil, app, complet, app, ultim, event,...
4        [2048, game, funnysimpl, addict, number, puzzl...
5        [2048, game, tip, trick, 1., take, slow, 2048,...
7            [2048, singleplay, slide, block, puzzl, game]
8        [2048, super, addict, brain, game, tri, merg, ...
9        [2048, play, gray, 44, grid, number, tile, sli...
10       [fun, game, must, gather, number, 2048, slide,...
11       [addict, puzzl, game, join, tile, get, 2048, t...
12       [introduc, simpl, fun, yet, addict, puzzl, gam...
13       [introduc, 2048, gamesbox, easi, fast, applic,...
14       [join, number, get, 2048, tile, swipe, move, t...
15       [perfect, 2048, puzzl, game, android, enjoy, 2...
16       [version, 2048, record, yuor, best, score, gam...
17       [2048, app, fun, addict, veri, simpl, puzzl, g...
18       [2048, app, fun, addict, veri, simpl, puzzl, g.

In [63]:
sum(b_list)

39

### Fill the result matrix 

In [90]:
def occurence(token,df):
    o1,o2,o3,o4=0,0,0,0
    
    o1 = sum([token in token_list for token_list in df[df['level']==1]['tokens']])
    o2 = sum([token in token_list for token_list in df[df['level']==2]['tokens']])
    o3 = sum([token in token_list for token_list in df[df['level']==3]['tokens']])
    o4 = sum([token in token_list for token_list in df[df['level']==4]['tokens']])
    return o1,o2,o3,o4
    
    

In [91]:
def fill(row):
    o1,o2,o3,o4 = occurence(row['token'],df)
    
    row['1'] = o1
    row['2'] = o2
    row['3'] = o3
    row['4'] = o4
    
    return row


In [92]:
result = result.apply(fill,axis=1)

In [93]:
result.to_csv('strong_words_distribution.csv')

### Find strong words in each app discription

In [94]:
def find(row):
    row['strong_tokens'] = [w for w in lst if w in row['tokens']]
    row['#strong_tokens'] = len(row['strong_tokens'])
    return row

In [95]:
df = df.apply(find,axis =1 )

In [96]:
df.to_csv('Andriod_have_strong_tokens.csv')

In [5]:
df = pd.read_excel('Andriod_have_strong_tokens0314.xlsx')

In [7]:
df

,name,sub_category,maturity_rating,maturity_rating_reason,download_num,developer,developer_address,price,datePublished,version,...,app_url,icon_url,description,isEnglish,new description,tokens,word_count,level,strong_tokens,#strong_tokens
1,101,Card,Everyone,NaN,"50,000 - 100,000",Frigate Studios,NaN,NaN,2016-12-01,1.23,...,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/mvneCoXs-pg2n...,"There are many variations of the basic game, a...",1,"there are many variations of the basic game, a...","[u'mani', u'variat', u'basic', u'game', u'numb...",42,1.0,[],0
2,2017,Adventure,Everyone,NaN,10 - 50,med dev,NaN,NaN,2016-12-29,1,...,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/-r9zhCalegAqw...,Hello you in your game Bunny helped Collect co...,1,hello you in your game bunny helped collect co...,"[u'hello', u'game', u'bunni', u'help', u'colle...",10,1.0,[],0
3,2017,Sports,Everyone,NaN,100 - 500,MYLAPS Experience Lab,Overschiestraat 65 1062 XD Amsterdam The Nethe...,NaN,2017-02-10,1,...,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/F_TGml00P7NBn...,The 2017 mobile app is the most complete app f...,1,the 2017 mobile app is the most complete app f...,"[u'2017', u'mobil', u'app', u'complet', u'app'...",39,1.0,[],0
4,2048,Puzzle,Everyone,NaN,50 - 100,prank call puzzle,NaN,NaN,2016-08-12,1,...,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/hVrtRVeoy_-yk...,2048 game is a funnysimple and addictive numbe...,1,2048 game is a funnysimple and addictive numbe...,"[u'2048', u'game', u'funnysimpl', u'addict', u...",101,1.0,[],0
5,2048,Puzzle,Everyone,NaN,10 - 50,iCare Edutainment,"D05, 28/F, TML Tower, No. 3 Hoi Shing Road, Ts...",NaN,2017-02-05,1.3,...,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/PqkgGplcxr3qF...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,1,2048 game tips & tricks 1. take it slow 2048 i...,"[u'2048', u'game', u'tip', u'trick', u'1.', u'...",145,1.0,[],0
6,2048,Puzzle,Unrated,Warning Š—– content has not yet been rated. Un...,"10,000 - 50,000",NG-TEAM,NaN,NaN,2014-06-08,2.2,...,https://play.google.com/store/apps/details?id=...,http://lh4.ggpht.com/IHXcAk-DG1WmAJhVNcUT_9O2E...,2048 is a simple but very attractive math puzz...,1,2048 is a simple but very attractive math puzz...,"[u'2048', u'simpl', u'veri', u'attract', u'mat...",60,NaN,[],0
7,2048,Puzzle,Everyone,NaN,100 - 500,hciLab,University of Stuttgart Human-Computer Interac...,NaN,2017-02-27,0.4,...,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/R-ildyLUXKs6I...,2048 is a singleplayer sliding block puzzle game.,1,2048 is a singleplayer sliding block puzzle game.,"[u'2048', u'singleplay', u'slide', u'block', u...",6,1.0,[],0
8,2048,Puzzle,Everyone,NaN,"10,000 - 50,000",Bazinga!,"No. 477 , Aomen Road Putuo Area,Shanghai,China",NaN,2014-06-21,1.02,...,https://play.google.com/store/apps/details?id=...,http://lh4.ggpht.com/mF_LE80-JtYPh4QhdlqTQ7xxe...,"2048 is a super addictive brain game , try to ...",1,"2048 is a super addictive brain game , try to ...","[u'2048', u'super', u'addict', u'brain', u'gam...",145,1.0,[],0
9,2048,Strategy,Everyone,NaN,"50,000 - 100,000",WANGJIE,10 Rue Jean Baptiste Charcot,NaN,2016-04-29,1.3,...,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/V6BDyrFCGp_tj...,"2048 is played on a gray 44 grid, with numbere...",1,"2048 is played on a gray 44 grid, with numbere...","[u'2048', u'play', u'gray', u'44', u'grid', u'...",90,1.0,[],0
10,2048,Puzzle,Everyone,NaN,100 - 500,App Sonic,"Jiron Trujillo 123, Surco, Lima, Per’_.",NaN,2016-07-06,4.0.0,...,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/iKKtouN22sjr-...,A fun game where you must gather the same numb...,1,a fun game where you must gather the same numb...,"[u'fun', u'game', u'must', u'gather', u'number...",18,1.0,[],0


In [97]:
df.columns

Index([u'name', u'sub_category', u'maturity_rating', u'maturity_rating_reason',
       u'download_num', u'developer', u'developer_address', u'price',
       u'datePublished', u'version', u'review_score', u'review_num',
       u'app_url', u'icon_url', u'description', u'isEnglish',
       u'new description', u'tokens', u'word_count', u'level',
       u'strong_tokens', u'#strong_tokens'],
      dtype='object')

In [8]:
df2 = df[df['#strong_tokens']!=0]

In [9]:
df3 = df2[['name','sub_category','maturity_rating','maturity_rating_reason',
                              'app_url','description','word_count','strong_tokens','#strong_tokens']]

In [10]:
df4 = df3[df3['maturity_rating']=='Everyone']

In [11]:
len(df4)

8388

In [12]:
df4.to_excel('Strong_Voc_Suspecious_app.xlsx')